In [2]:
%pip install langfuse
%pip install langchain langgraph langchain_openai langchain_community
%pip install --upgrade --quiet  yandexcloud yandex-cloud-ml-sdk


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Заполняем словарь документов

In [3]:
documents = [
    {
        "id": 166213,
        "title": "Изменить формат общего шаблона excel при скачивании файлов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/"
    },

    {
        "id": 151543,
        "title": "Реализовать страницу со списком акселераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/"
    },

    {
        "id": 196677,
        "title": "[Bug][Major] Доработать статусы заявок в Акселераторах",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/196677/"
    },

    {
        "id": 181744,
        "title": "Реализация группы прав \"Менеджер Акселератора партнер\"",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/181744/"
    },

    {
        "id": 190897,
        "title": "Доработка кабинета Акселератора, добавление внешних модераторов",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/190897/"
    },

    {
        "id": 184464,
        "title": "Доработка ролевой модели в Акселерации (Демо)",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/184464/"
    },

    {
        "id": 187834,
        "title": "Добавление рассылки видеозаписи прошедшего мероприятия",
        "url": "https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/"
    },
]

обогащаем documents текстами задач

In [4]:
import os

# Добавляем поле context с текстом из .md файлов
for doc in documents:
    file_name = f"{doc['id']}.md"
    file_path = os.path.join("tasks", file_name)

    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            doc["context"] = f.read()
    else:
        doc["context"] = ""

    # Создаем поле search как объединение title и context
    doc["search"] = f"{doc['title']} {doc['context']}"



In [5]:
import pandas as pd  # если нет импорта пандаса, то остальной код не имеет смысла

# создаем из наших документов датафрейм
df = pd.DataFrame(documents)
df.head()

,id,title,url,context,search
0,166213,Изменить формат общего шаблона excel при скачи...,https://inside.notamedia.ru/company/personal/u...,Требуется изменить формат excel общего шаблона...,Изменить формат общего шаблона excel при скачи...
1,151543,Реализовать страницу со списком акселераторов,https://inside.notamedia.ru/company/personal/u...,"1. Необходимо сделать активной кнопку ""Акселер...",Реализовать страницу со списком акселераторов ...
2,196677,[Bug][Major] Доработать статусы заявок в Аксел...,https://inside.notamedia.ru/company/personal/u...,Стенд: все площадки\n\n1) Кейс 1\n- Создать Ак...,[Bug][Major] Доработать статусы заявок в Аксел...
3,181744,"Реализация группы прав ""Менеджер Акселератора ...",https://inside.notamedia.ru/company/personal/u...,"Требуется реализовать новую группу прав ""Менед...","Реализация группы прав ""Менеджер Акселератора ..."
4,190897,"Доработка кабинета Акселератора, добавление вн...",https://inside.notamedia.ru/company/personal/u...,Требуется в кабинете Акселератора доработать д...,"Доработка кабинета Акселератора, добавление вн..."


In [6]:
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

loader = DataFrameLoader(df, page_content_column='search')
documents = loader.load()

print(documents)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [7]:
# # создаем сплиттер документов, чтобы уложиться в лимит по токенам, в нашем случае это не очень полезный шаг

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(texts)

[Document(metadata={'id': 166213, 'title': 'Изменить формат общего шаблона excel при скачивании файлов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/166213/', 'context': 'Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'}, page_content='Изменить формат общего шаблона excel при скачивании файлов Требуется изменить формат excel общего шаблона при скачивании файлов. Изменить с формата "Веб-страница" https://pastenow.ru/GMPC8 на xls.\n\nОтчеты в xls в блоке Бизнес-образования https://pastenow.ru/GMPBC\n\nОценка: 1 р.д.'), Document(metadata={'id': 151543, 'title': 'Реализовать страницу со списком акселераторов', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/', 'context': '1. Необходимо сделать активной кнопку "Акселерация" в разделе Бизнес-комп

In [8]:
import os

YC_FOLDER_ID = os.getenv("folder_id")
YC_API_KEY = os.getenv("ya_token")


In [9]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings

# Создаем модель для генерации эмбеддингов
yandex_embeddings = YandexGPTEmbeddings(model="doc", folder_id=YC_FOLDER_ID, iam_token=YC_API_KEY)


Создаем индекс в бд clickhouse

Запускаем бд в docker контейнере

In [ ]:
%%sh
docker run -d --rm --network=host --name some-clickhouse-server --ulimit nofile=262144:262144 clickhouse

Проверяем доступ и создает таблицу если нет

In [1]:
%pip install clickhouse-connect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [clickhouse-connect]
Note: you may need to restart the kernel to use updated packages.


In [25]:
import clickhouse_connect

client = clickhouse_connect.get_client(host="localhost", username="default", password="")
client.command("DROP TABLE IF EXISTS test_clickhouse")

client.command(f"""
CREATE TABLE test_clickhouse (
    id UInt64,
    text String,
    title String,
    url String,
    embedding Array(Float32)
) ENGINE = MergeTree() ORDER BY id
""")

In [26]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(
    folder_id=YC_FOLDER_ID,
    auth=YC_API_KEY,
)

Заполняем бд

In [31]:
for doc in (texts):

    # Получаем поля из документа
    id = doc.metadata.get("id", 0)
    url = doc.metadata.get("url", "")
    title = doc.metadata.get("title", "")
    text = doc.page_content  # Добавляем текст документа

    # Генерируем эмбеддинг
    doc_model = sdk.models.text_embeddings("doc")
    doc_embeddings = doc_model.run(text)

    embedding_array = doc_embeddings.embedding


    # Вставляем строку в таблицу
    client.insert(
        "test_clickhouse",
        [
            (id, text, title, url, embedding_array)
        ]
    )


Запрос:

In [32]:
query = 'Какие поля есть в карточке мероприятия?'

Векторизуем запрос

In [34]:
query_model = sdk.models.text_embeddings("query")
query_embedding = query_model.run(query)

print(query_embedding)

TextEmbeddingsModelResult(embedding=(-0.0085601806640625, -0.0162353515625, -0.06597900390625, 0.005767822265625, -0.041748046875, -0.03179931640625, 0.0318603515625, -0.10015869140625, -0.0105438232421875, -0.00682830810546875, -0.035919189453125, -0.0006852149963378906, 0.0469970703125, -0.054107666015625, 0.05126953125, 0.022125244140625, 0.046844482421875, -0.1373291015625, -0.10589599609375, -0.11102294921875, 0.058074951171875, 0.031951904296875, 0.0770263671875, -0.034088134765625, 0.0623779296875, 0.006072998046875, -0.028167724609375, -0.036376953125, 0.09661865234375, -0.040283203125, 0.01250457763671875, -0.004863739013671875, -0.07672119140625, 0.03460693359375, 0.00649261474609375, -0.0836181640625, -0.051361083984375, 0.0281982421875, 0.041839599609375, -0.11383056640625, -0.1353759765625, -0.1136474609375, 0.0227203369140625, -0.0209808349609375, -0.0265960693359375, 0.057952880859375, 0.018280029296875, 0.0266265869140625, 0.046966552734375, 0.007965087890625, -0.016845

Проверяем индекс в clickhouse

In [40]:

query_str = ",".join(map(str, query_embedding))
result = client.query(f"""
SELECT id, cosineDistance(embedding, [{query_str}]) AS dist
FROM test_clickhouse
ORDER BY dist ASC
LIMIT 2
""")

print("🔍 ClickHouse:", [row[0] for row in result.result_rows])

🔍 ClickHouse: [187834, 151543]


Выносим код в функцию

In [41]:
def search_similar_documents(query: str):
    """
    Поиск похожих документов в ClickHouse с использованием косинусного расстояния

    Parameters:
        client (clickhouse_connect.Client): Подключённый клиент к ClickHouse
        query_embedding (list[float]): Вектор-эмбеддинг запроса

    Returns:
        list[int]: Список ID найденных документов
    """

    query_model = sdk.models.text_embeddings("query")
    query_embedding = query_model.run(query)

    # Преобразуем эмбеддинг в строку для SQL-запроса
    query_str = ",".join(map(str, query_embedding))

    # Выполняем запрос с расчётом косинусного расстояния
    result = client.query(f"""
        SELECT id, title, url, text, cosineDistance(embedding, [{query_str}]) AS dist
        FROM test_clickhouse
        ORDER BY dist ASC
        LIMIT 2
    """)

    # Формируем список словарей с данными документов
    similar_docs = [
        {
            "id": row[0],
            "title": row[1],
            "url": row[2],
            "text": row[3]
        }
        for row in result.result_rows
    ]

    return similar_docs

print(search_similar_documents(query))

[{'id': 187834, 'title': 'Добавление рассылки видеозаписи прошедшего мероприятия', 'url': 'https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/', 'text': 'Добавление рассылки видеозаписи прошедшего мероприятия Требуется добавить элемент по управлению рассылками видеозаписи прошедшего мероприятия\n\n1. В карточку мероприятия добавить возможность прикрепление обработанной видеозаписи (смонтированной) и записи после мероприятия (автоматическая запись трансляции)\nДоработать карточку мероприятия, добавить:\n1. 1. Чек-бокс "Добавить видеозапись после завершения мероприятия (автоматическая запись трансляции)";\n1. 2. Поле "Добавить видеозапись после завершения мероприятия (Обработанная видеозапись)"\nСтандартное поле для загрузки видео (как в акселераторе и видеолекциях), добавление через загрузку с ПК или медиабиблиотеку\nСнимок экрана 2023-02-11 в 16.57.29.png\n\n\n1. 3. Новые поля разместить после поля: Количество зарегистрированных пользователей: 1'}, {'id': 151

Создаем модель

In [43]:
from langchain_community.llms import YandexGPT

llm = YandexGPT(
    folder_id=YC_FOLDER_ID,
    iam_token=YC_API_KEY,
    model="yandexgpt",
    temperature=0.3,
    max_tokens=500
)

Подключаем промпт

In [46]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Создаем шаблон промпта с полями из метаданных
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""Ответь на вопрос, используя следующие документы:

Контекст:
{context}

Вопрос:
{question}

Ответ:"""
)

# цепочка с кастомным промтом
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

relevants = search_similar_documents(query)

# Формируем контекст и метаданные
context = "\n\n".join([doc['text'] for doc in relevants])

# Отправляем запрос в LLMChain
result = llm_chain.invoke({
    "question": query,
    "context": context
})

print("Вопрос:")
print(query)

print("Ответ:")
print(result["text"])

print(f"\n\nПодробнее в задачах:\n")
for doc in relevants:
    print(f"{doc["title"]}\n{doc["url"]}\n")


Вопрос:
Какие поля есть в карточке мероприятия?
Ответ:
В карточке мероприятия есть следующие поля:
* Краткое описание — анонс.
* Дата старта сбора заявок.
* Дата окончания сбора заявок.
* Продление сбора заявок (чек-бокс).
* Поле для даты окончания сбора заявок (в случае активации чекбокса «Продление сбора заявок»).
* Поле для загрузки смонтированной видеозаписи.
* Чек-бокс «Добавить видеозапись после завершения мероприятия (автоматическая запись трансляции)».
* Поле для загрузки автоматической записи трансляции после завершения мероприятия.
* Поле «Количество зарегистрированных пользователей».


Подробнее в задачах:

Добавление рассылки видеозаписи прошедшего мероприятия
https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/187834/

Реализовать страницу со списком акселераторов
https://inside.notamedia.ru/company/personal/user/4658/tasks/task/view/151543/

